In [1]:
import pandas as pd
import numpy as np
import glob, os, sys
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='notebook', style = 'whitegrid', font_scale = 1.3)

In [2]:
sys.path.append('..')
from modules.plotting_metrics import PlotMetric

In [19]:
prot_name = 'cdk2'
# Este es el DF que no tiene aun afgregados los valores AUC de las conformaciones
path_to_json_file = glob.glob((os.path.join('..', 'data', 
                        F'TABLA_MTDATA_{prot_name.upper()}_*_crys_LIGS_INFO_LABELS.json')))[0]
                      
df_prot = pd.read_json(path_to_json_file)

In [20]:
# Extract just the interesting features
df_prot.columns
columns_ = ['Labels_conf', 'Inhib', 'Inhib_mass']
df_CDK2 = df_prot[columns_]

#### MDS info

In [4]:
# Read the MDS resutls
import pickle

# Pocket MDS
path_mds_obj_pk = os.path.join('..', 'data', 'trajectory_analysis', 'cMDS_Pocket_402_obj.pyobj')
with open(path_mds_obj_pk, 'rb') as f:
    mds_pocket_402 = pickle.load(f)
    
# Pisani MDS
path_mds_obj = os.path.join('..', 'data', 'trajectory_analysis', 'cMDS_Pisani_402_obj.pyobj')
with open(path_mds_obj, 'rb') as f:
    mds_pisani_402 = pickle.load(f)
    
a = mds_pisani_402[0][:2]
b = mds_pocket_402[0][:2]

c = np.concatenate((a, b))

np.savetxt('./mds_subspaces_pis_pk.csv', c, delimiter = ',')

In [6]:
# Helper function
%run ./helper_functions_S6.ipynb

In [15]:
score_type = 'Dk_lef'
df_swarm_dklef = pd.DataFrame()

for db_name in scores_dic:
    _X = scores_dic[db_name][score_type]['X']
    _y = scores_dic[db_name][score_type]['y']
    auc_scores = _X.apply(lambda x: roc_auc_score(y_true =_y, 
                                  y_score = -1 * x), axis = 0)
    score_type_ = score_type.lower().replace('_', '')
    df_swarm_dklef[db_name + '_vrd_' + score_type_] = auc_scores

score_type = 'Dk_sc'
df_swarm_dksc = pd.DataFrame()

for db_name in scores_dic:
    _X = scores_dic[db_name][score_type]['X']
    _y = scores_dic[db_name][score_type]['y']
    auc_scores = _X.apply(lambda x: roc_auc_score(y_true =_y, 
                                  y_score = -1 * x), axis = 0)
    score_type_ = score_type.lower().replace('_', '')
    df_swarm_dksc[db_name + '_vrd_' + score_type_] = auc_scores
    

In [25]:
x = pd.concat([df_CDK2, df_swarm_dksc, df_swarm_dklef], axis = 1)

In [26]:
x.to_csv('./df_pdb_cdk2_features_and_scores.csv')